In [ ]:
import Pkg
Pkg.activate("..")

In [ ]:
Pkg.status()

In [ ]:
include("burgers_common.jl")

In [ ]:
import BurgersEquation
import CSV
import DataFrames
import Random

import ImplicitAD as IAD

In [ ]:
using Plots
using Plots.Measures
using Statistics

In [ ]:
# function svd_dimensions(ngrid)
#     k = log2(ngrid)
#     m = 2^Int(ceil(k/2))
#     n = 2^Int(floor(k/2))
#     return (m,n)
# end

# x = rand(2^10)
# (m,n) = svd_dimensions(length(x))
# @time xsvd = IAD.SVDVector(x, m, n, 5)
# @show Base.summarysize(x)
# @show Base.summarysize(xsvd)
# ;

# Helper Functions

In [ ]:
function reduce_field(result_df, field; keep_target=true)

    f_df = DataFrames.DataFrame(
        :AD=>Symbol[],
        :GridSize => Int[],
        :Target => Symbol[],
        :Mean => Float64[],
        :Median => Float64[],
        :Min => Float64[],
        :Max => Float64[],
        :Std => Float64[],
    )

    my_group = keep_target ? [:AD, :GridSize, :Target] : [:AD, :GridSize]

    for (k, df) in enumerate(DataFrames.groupby(result_df, my_group))
        ad = first(unique(df[:,:AD]))
        gs = first(unique(df[:,:GridSize]))
        target = first(unique(df[:,:Target]))
        
        row_dict = Dict(
            :AD => ad,
            :GridSize => gs,
            :Target => target,
            :Mean => mean(df[:,field]),
            :Median => median(df[:,field]),
            :Min => minimum(df[:,field]),
            :Max => maximum(df[:,field]),
            :Std => std(df[:,field]),
        )
        push!(f_df, row_dict)
    end

    if !keep_target
        DataFrames.select!(f_df, DataFrames.Not(:Target))
    end

    return f_df

end

In [ ]:
function reduce_memory(result_df; keep_target=true)
    return reduce_field(result_df, :MaxRSS; keep_target)
end

In [ ]:
function reduce_time(result_df; keep_target=true)
    return reduce_field(result_df, :Time; keep_target)
end

In [ ]:
function svd_mem(nsv, ngrid)
    (m,n) = svd_dimensions(ngrid)
    return nsv * (m + n + 1)
end

# Computational Test Results

In [ ]:
function make_base_name(ad, nx, run)
    return "large_scale_burger_solution_$(ad)_n$(nx)_s$(run)"
end

function make_file_name(ad, nx, run)
    return make_base_name(ad, nx, run) * ".txt"
end

function parse_results_file(fname::AbstractString)

    mrstr = read(fname, String)

    memreg = r"Max. RSS: *([0-9]+\.[0-9]*) MiB"
    m = match(memreg, mrstr)
    mem_use = -1.0
    if m !== nothing
        mem_use = parse(Float64, m[1])
    else
        error("Failed to find Max RSS memory")
    end

    treg = r"Seconds: *([0-9]+\.[0-9]*)"
    m = match(treg, mrstr)
    time = -1.0
    if m !== nothing
        time = parse(Float64, m[1])
    else
        error("Failed to find run time")
    end

    ncreg = r"Number of Calls: *([0-9]+)"
    m = match(ncreg, mrstr)
    ncalls = 0
    if m !== nothing
        ncalls = parse(Int, m[1])
    else
        ncalls = 1
    end

    creg = r"Converged: *([a-z]+)"
    m = match(creg, mrstr)
    converged = false
    if m !== nothing
        converged = parse(Bool, m[1])
    end

    # greg = r"Gradient Residual: (/[+\-]?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+\-]?\d+)?/)"
    greg = r"Gradient Residual: ([0-9\.+\-eE]+)"
    m = match(greg, mrstr)
    gres = -1.0
    if m !== nothing
        gres = parse(Float64, m[1])
    # else
    #     error("Failed to find gradient residual!!")
    end

    # xreg = r"x Absolute Change: (/[+\-]?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+\-]?\d+)?/)"
    xreg = r"x Absolute Change: ([0-9\.+\-eE]+)"
    m = match(xreg, mrstr)
    xres = -1.0
    if m !== nothing
        xres = parse(Float64, m[1])
    # else
    #     error("Failed to find x change residual!!")
    end
    
    return (mem_use, time, ncalls, converged, gres, xres)

end

function parse_memory_results_to_dataframe(
        resdir::AbstractString,
        ad_methods::Vector,
        grid_sizes::Vector,
        targets::Vector,
        nruns::Integer,
        optimization::Bool,
)

    my_ads = Vector{Symbol}()
    tars = Vector{Symbol}()
    my_run = Vector{Int}()
    size = Vector{Int}()
    max_rss = Vector{Float64}()
    times = Vector{Float64}()
    ncalls = Vector{Int}()
    converged = Vector{Bool}()
    residuals = Vector{Float64}()
    xdeltas = Vector{Float64}()

    # not_found = Dict()

    for target in targets
        rdir = joinpath(resdir, target)
        for ad in ad_methods
            for nx in grid_sizes
                k = Int(log2(nx))
                for run in 1:nruns
                    fn = joinpath(rdir, make_file_name(ad, nx, run))
                    #### Optimization Excludes ####
                    if optimization &&
                        (k > 11 && ad == :finitediff
                        || k > 12 && ad == :forward
                        || k > 12 && ad == :reverse
                        || k > 12 && ad == :dirreverse
                    )
                        continue
                    end
                    #### Gradient Evaluation Excludes ####
                    if !optimization &&
                        (k > 13 && ad == :finitediff
                        || k > 14 && ad == :forward
                        || k > 13 && ad == :reverse
                        || k > 13 && ad == :dirreverse
                    )
                        continue
                    end
                    try
                        (mr, tr, nc, cnv, res, dx) = parse_results_file(fn)
                        push!(my_ads, ad)
                        push!(tars, Symbol(target))
                        push!(my_run, run)
                        push!(size, nx)
                        push!(max_rss, mr)
                        push!(times, tr)            
                        push!(ncalls, nc)
                        push!(converged, cnv)
                        push!(residuals, res)
                        push!(xdeltas, dx)
                    catch e
                        if isa(e, SystemError)
                            println("For target $(target), failed to find file: ", make_file_name(ad, nx, run))
                        else
                            rethrow(e)
                        end
                    end
                end
            end
        end
    end

    df = DataFrames.DataFrame(Dict(
            :AD => my_ads,
            :Target => tars,
            :Run => my_run,
            :GridSize => size,
            :MaxRSS => max_rss,
            :Time => times,
            :NumCalls => ncalls,
            :Converged => converged,
            :Residual => residuals,
            :dx => xdeltas,
        ))
    return df

end

In [ ]:
const GRAD_RESIDUAL = 1e-4;
# machine = "local"
machine = "kestrel"
compute = "optimization"
results_dir = joinpath(@__DIR__, "results", machine, compute)
if compute == "optimization"
    ad_modes = [:svdreverse, :impreverse]
    grid_sizes = 2 .^ collect(12:12)
else
    ad_modes = [:forward, :reverse, :finitediff, :svdreverse, :impreverse, :dirreverse]
    grid_sizes = 2 .^ collect(4:16)
end
# ad_modes = [:forward, :finitediff, :svdreverse, :impreverse]
targets = string.([:sin, :cliff, :weierstrass])
# targets = string.([:sin])
# grid_sizes = 2 .^ collect(4:14)

nruns = 10
@show results_dir
res_df = parse_memory_results_to_dataframe(results_dir, 
    ad_modes, grid_sizes, targets, nruns, compute == "optimization")
@show sum(res_df[:,:Time])
@show sum(res_df[:,:Time]) / 3600.0
;

In [ ]:
res_df

In [ ]:
# idx = res_df[:,:AD] .== :forward
# @show res_df[idx,:]
# res_df[idx, :Time] ./ 3600.0

In [ ]:
# idx = res_df[:,:AD] .== :reverse
# @show res_df[idx,:]
# res_df[idx, :Time] ./ 3600.0

In [ ]:
# idx = res_df[:,:AD] .== :impreverse
# @show res_df[idx,:]
# res_df[idx, :Time] ./ 3600.0

In [ ]:
# idx = res_df[:,:AD] .== :svdreverse
# @show res_df[idx,:]
# res_df[idx, :Time] ./ 3600.0

In [ ]:
# idx = res_df[:,:AD] .== :finitediff
# @show res_df[idx,:]
# res_df[idx, :Time] ./ 3600.0

In [ ]:
if compute == "optimization"
    idx = res_df[:,:Converged] .== false
    @show sum(idx)
end

In [ ]:
if compute == "optimization"
    for (k, df) in enumerate(DataFrames.groupby(res_df, [:AD, :Target]))
        # @show df
        ad = first(unique(df[:,:AD]))
        target = first(unique(df[:,:Target]))
        idx = df[:,:Converged] .== true
        println("**** AD: $(ad) Target: $(target) ****")
        frac = sum(idx) / size(df, 1)
        println(sum(idx), " of ", size(df,1), " ($(frac)) converged")
        idx = df[:,:Residual] .< GRAD_RESIDUAL
        frac = sum(idx) / size(df, 1)
        println(sum(idx), " of ", size(df, 1), " ($(frac)) below gradient residual")
    end
end

In [ ]:
if compute == "optimization"
    ad_df = DataFrames.groupby(res_df, :AD)
    p = plot(;
        legend=:outerbottom,
        legend_column=2,
        title="Optimization Residuals",
        ylabel="|∇f|_∞",
        xlabel="Grid size",
        yscale=:log10,
        # ylim=(10^-5, 10^1),
        # xscale=:log2,
        # xticks=grid_sizes
        bottommargin=-7mm,
    )
    df = first(ad_df)
    # gs = unique(df[:,:GridSize])
    plot!(p, fill(GRAD_RESIDUAL, size(df,1)), color=:red, style=:dash, label=nothing)
    for (k,df) in enumerate(ad_df)
        # println(df)
        method = df[1,:AD]
        # gs = df[:,:GridSize]
        residuals = df[:,:Residual]
        # plot!(p, grid_sizes, residuals, color=k, label=string(method))
        # scatter!(p, gs, residuals, color=k, label=string(method))
        scatter!(p, residuals, color=k, label=string(method))
    end
    display(p)
end

In [ ]:
if compute == "optimization"
    p = plot(;
        # legend=:outerbottom,
        # legend_column=2,
        legend=true,
        title="Initial Conditions",
        # ylabel="|∇f|_∞",
        # xlabel="Grid size",
        # yscale=:log10,
        # ylim=(10^-5, 10^1),
        # xscale=:log2,
        # xticks=grid_sizes
        # bottommargin=-7mm,
    )
    opt_nx = first(unique(df[:,:GridSize]))
    xgrid = BE.space_grid(opt_nx)
    # umax = 0.505
    # umin = 0.495
    umax = 0.55
    umin = 0.45
    for k in 1:nruns
        rng = Random.MersenneTwister(k)
        u0 = (umax - umin) .* randn(rng, opt_nx) .+ 0.5 * (umax + umin)
        scatter!(p, xgrid, BE.expand_solution(u0), color=k, label=k)
    end
    display(p)
end

In [ ]:
0.505 - 0.495

In [ ]:
mem_df = reduce_memory(res_df; keep_target=false)
p = plot(;
    legend=:topleft,
    title="Max Memory Footprint",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log2,
    yticks=10.0 .^ (1.0:0.5:6.0),
    xscale=:log2,
)
for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
    # println(df)
    method = df[1,:AD]
    gs = df[:,:GridSize]
    mem_res = df[:,:Min] .- 720
    plot!(p, gs, mem_res, color=k, label=string(method))
    # plot!(p, grid_sizes, df[:,:Min], color=k, label=nothing, style=:dash)
    # plot!(p, grid_sizes, df[:,:Max], color=k, label=nothing, style=:dash)
    scatter!(p, gs, mem_res, color=k, label=nothing)
end
display(p)

In [ ]:
mem_df = reduce_memory(res_df; keep_target=true)
p = plot(;
    legend=:topleft,
    title="Max Memory Footprint STD",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log10,
    yticks=10.0 .^ (0.0:0.5:5.0),
    xscale=:log2,
)
for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
    # println(df)
    method = df[1,:AD]
    for rdf in DataFrames.groupby(df, :Target)
        target = rdf[1,:Target]
        gs = rdf[:,:GridSize]
        mem_res = rdf[:,:Std]
        plot!(p, gs, mem_res, color=k, label=string(method))
        scatter!(p, gs, mem_res, color=k, label=nothing)
    end
end
display(p)

In [ ]:
# idx = (mem_df[:,:AD] .== :svdreverse) .& (mem_df[:,:GridSize] .== 2048)
idx = mem_df[:,:GridSize] .== 2048
mem_df[idx, :]

In [ ]:
mem_df = reduce_memory(res_df; keep_target=false)
p1 = plot(;
    legend=:topleft,
    title="Max Memory Footprint STD",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log10,
    yticks=10.0 .^ (0.0:0.5:5.0),
    xscale=:log2,
)
p2 = plot(legend=:topleft, xscale=:log2)
for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
    # println(df)
    method = df[1,:AD]
    gs = df[:,:GridSize]
    mem_res = df[:,:Std]
    plot!(p1, gs, mem_res, color=k, label=string(method))
    scatter!(p1, gs, mem_res, color=k, label=nothing)

    mem_res = df[:,:Std] ./ df[:,:Min]
    plot!(p2, gs, mem_res, color=k, label=string(method))
    scatter!(p2, gs, mem_res, color=k, label=nothing)
    mem_res = df[:,:Std] ./ df[:,:Max]
    plot!(p2, gs, mem_res, color=k, label=nothing)
    scatter!(p2, gs, mem_res, color=k, label=nothing)
end
p = plot(p1, p2, size=(1000,400))
display(p)

In [ ]:
# ad_df = DataFrames.groupby(res_df, [:AD, :Run])
p = plot(;
    legend=:topleft,
    title="Max Memory Footprint",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log10,
    yticks=10.0 .^ (2.8:0.1:5.0),
    xscale=:log2,
)
for (k,df) in enumerate(DataFrames.groupby(res_df, :AD))
    # println(df)
    make_label = true
    for rdf in DataFrames.groupby(df, [:Run,:Target])
        # @show rdf
        method = rdf[1,:AD]
        gs = rdf[:,:GridSize]
        mem_res = rdf[:,:MaxRSS]
        if make_label
            label = string(method)
            make_label = false
        else
            label = nothing
        end
        plot!(p, gs, mem_res, color=k, label=label)
        scatter!(p, gs, mem_res, color=k, label=nothing)
    end
end
display(p)

In [ ]:
# ad_df = DataFrames.groupby(res_df, [:AD, :Run])
mem_df = reduce_memory(res_df; keep_target=false)
p = plot(;
    legend=:topleft,
    title="Max Memory Footprint",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log10,
    yticks=10.0 .^ (2.8:0.1:5.0),
    xscale=:log2,
)
for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
    
    method = df[1,:AD]

    if method != :svdreverse && method != :impreverse
        continue
    end

    gs = df[:,:GridSize]
    mem_res = df[:,:Min]
    
    plot!(p, gs, mem_res, color=k, label=string(method))
    scatter!(p, gs, mem_res, color=k, label=nothing)

end

display(p)

In [ ]:
log10(720.0)

In [ ]:
minimum(res_df[:,:MaxRSS])

In [ ]:
# ad_df = DataFrames.groupby(res_df, [:AD, :Run])
mem_df = reduce_memory(res_df; keep_target=false)
p = plot(;
    legend=:topleft,
    title="Max Memory Footprint",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log10,
    yticks=10.0 .^ (2.8:0.1:5.0),
    xscale=:log2,
    xlim=(2^10, 2^17),
)

for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
    
    method = df[1,:AD]

    if method != :svdreverse && method != :impreverse
        continue
    end

    gs = df[:,:GridSize]
    mem_res = df[:,:Median]
    
    plot!(p, gs, mem_res, color=k, label=string(method))
    scatter!(p, gs, mem_res, color=k, label=nothing)

end

# gs = sort(unique(mem_df[:,:GridSize]))
# plot!(p, gs, sqrt.(gs) .+ 1000, label=nothing)
# plot!(p, gs, gs, label=nothing)

display(p)

In [ ]:
for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
    ad = first(df[:,:AD])
    ymb = log2.(df[:,:Median])
    xlog = log2.(df[:,:GridSize])
    slopes = (ymb[2:end] .- ymb[1:end-1]) ./ (xlog[2:end] .- xlog[1:end-1])
    @show (ad, slopes)
end

In [ ]:
# ad_df = DataFrames.groupby(res_df, [:AD, :Run])
mem_df = reduce_memory(res_df; keep_target=false)
p = plot(;
    legend=:topleft,
    title="Max Memory Footprint",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log10,
    yticks=10.0 .^ (2.8:0.1:5.0),
    xscale=:log2,
)
for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
    
    method = df[1,:AD]

    if method != :forward && method != :finitediff
        continue
    end

    gs = df[:,:GridSize]
    mem_res = df[:,:Min]
    
    plot!(p, gs, mem_res, color=k, label=string(method))
    scatter!(p, gs, mem_res, color=k, label=nothing)

end

display(p)

In [ ]:
mem_df

In [ ]:
mem_df[mem_df[:,:AD] .== :svdreverse, :]

In [ ]:
mem_df[mem_df[:,:AD] .== :impreverse, :]

In [ ]:
const OVERHEAD = 184
nsv_max = 5

tf = 1.0
cfl = 0.85
seed = 1

umax = 0.6
# umin = 0.495

est_grid_sizes = 6:17

mem_save_bytes = zeros(length(est_grid_sizes))
mem_save_bytes_nsv1 = zeros(length(est_grid_sizes))

all_grid_sizes = sort(union(grid_sizes, 2.0.^est_grid_sizes))

for (idx, k) in enumerate(est_grid_sizes)

    Nx = 2^k
    (m,n) = svd_dimensions(Nx)
    nsv = min(m, n, nsv_max)

    @show Nx, nsv

    (dt, Nt) = set_dt_nsteps(tf, umax, cfl, Nx)
    savings_per_step = (Nx - svd_mem(nsv, Nx)) * sizeof(Float64) - OVERHEAD
    total_savings = savings_per_step * Nt

    mem_save_bytes[idx] = total_savings > 0 ? total_savings : NaN

    savings_per_step = (Nx - svd_mem(1, Nx)) * sizeof(Float64) - OVERHEAD
    total_savings = savings_per_step * Nt
    mem_save_bytes_nsv1[idx] = total_savings > 0 ? total_savings : NaN

end

idx_svd = mem_df[:,:AD] .== :svdreverse
idx_imp = mem_df[:,:AD] .== :impreverse
# mem_diff = res_df[idx_imp, :MaxRSS] - res_df[idx_svd, :MaxRSS]
mem_diff = mem_df[idx_imp, :Median] - mem_df[idx_svd, :Median]
@show mem_diff
mem_diff[mem_diff .< 1] .= NaN
p = plot(xscale=:log2, yscale=:log10,
    xticks=all_grid_sizes, yticks=[10.0^k for k in -2.5:0.5:8.0],
    legend=:topleft, title="Memory Difference (MB)")
plot!(p, 2.0.^est_grid_sizes, mem_save_bytes ./ 1024^2, label="NSV 5")
plot!(p, 2.0.^est_grid_sizes, mem_save_bytes_nsv1 ./ 1024^2, label="NSV 1")
for field in [:Min, :Mean, :Median, :Max]
    mem_diff = mem_df[idx_imp, field] - mem_df[idx_svd, field]
    mem_diff[mem_diff .< 1e-6] .= NaN
    scatter!(p, grid_sizes, mem_diff, label=string(field))
end
display(p)

In [ ]:
nruns

In [ ]:
tf

In [ ]:
cfl

In [ ]:
grid_sizes

In [ ]:
est_grid_sizes

In [ ]:
# for (k,df) in enumerate(DataFrames.groupby(mem_df, :AD))
#     ad = first(df[:,:AD])
#     ymb = log2.(df[:,:Median])
#     xlog = log2.(df[:,:GridSize])
#     slopes = (ymb[2:end] .- ymb[1:end-1]) ./ (xlog[2:end] .- xlog[1:end-1])
#     @show (ad, slopes)
# end

In [ ]:
est_grid_sizes = 4:16
mem_save_bytes = zeros(length(est_grid_sizes))
mem_save_bytes_nsv1 = zeros(length(est_grid_sizes))
all_grid_sizes = sort(union(grid_sizes, 2.0.^est_grid_sizes))

per_step_mem_df = reduce_memory(res_df; keep_target=false)
per_step_mem_df[:,:MeanPerStep] = zeros(size(per_step_mem_df, 1))

for k in est_grid_sizes
    Nx = 2^k
    Nt_avg = 0.0
    for seed in 1:nruns
        rng = Random.MersenneTwister(seed)
        umax = 0.505
        umin = 0.495
        xmax = maximum((umax - umin) .* randn(rng, Nx) .+ 0.5 * (umax + umin))
        # xmax = maximum(x0)
        (dt, Nt) = set_dt_nsteps(tf, xmax, cfl, Nx)
        # @show (Nx, dt, Nt)
        Nt_avg += (Nt - Nt_avg) / seed
    end
    # @show (Nx, Nt_avg)
    idx = per_step_mem_df[:,:GridSize] .== Nx
    # per_step_mem_df[idx, :MeanPerStep] .= per_step_mem_df[idx,:Mean] ./ Nt_avg
    per_step_mem_df[idx, :MeanPerStep] .= (per_step_mem_df[idx,:Mean] .- 720.0) ./ Nt_avg
end
# per_step_mem_df
p = plot(
    # xlim=(2^8, 2^17),
    xscale=:log2,
    yscale=:log10,
    ylabel="Memory per Step (kB)",
)
for (k, df) in enumerate(DataFrames.groupby(per_step_mem_df, [:AD]))
    ad = first(df[:,:AD])
    if ad == :forward || ad == :finitediff || ad == :dirreverse
        continue
    end
    gs = df[:,:GridSize]
    mps = df[:,:MeanPerStep] * 1024
    plot!(p, gs, mps, color=k, label=string(ad))
    scatter!(p, gs, mps, color=k, label=nothing)
end
display(p)

In [ ]:
tm_df = reduce_time(res_df; keep_target=false)
p = plot(;
    legend=:topleft,
    title="Total Wall Clock Time",
    ylabel="Time(s)",
    xlabel="Grid size",
    yscale=:log10,
    xscale=:log2,
    # xticks=grid_sizes
)
for (k,df) in enumerate(DataFrames.groupby(tm_df, :AD))
    method = df[1,:AD]
    gs = df[:,:GridSize]
    t_res = df[:,:Min]
    plot!(p, gs, t_res, color=k, label=string(method))
    scatter!(p, gs, t_res, color=k, label=nothing)
end
display(p)

In [ ]:
tm_df = reduce_time(res_df; keep_target=false)
p1 = plot(;
    legend=:topleft,
    title="Total Wall Clock Time",
    ylabel="Time(s)",
    xlabel="Grid size",
    yscale=:log10,
    xscale=:log2,
    # xticks=grid_sizes
)
p2 = plot(legend=:topright, xscale=:log2)
for (k,df) in enumerate(DataFrames.groupby(tm_df, :AD))
    # println(df)
    method = df[1,:AD]
    gs = df[:,:GridSize]
    t_res = df[:,:Std]
    plot!(p1, gs, t_res, color=k, label=string(method))
    scatter!(p1, gs, t_res, color=k, label=nothing)

    t_res = df[:,:Std] ./ df[:,:Min]
    plot!(p2, gs, t_res, color=k, label=string(method))
    scatter!(p2, gs, t_res, color=k, label=nothing)
end
p = plot(p1, p2, size=(1000,400))
display(p)

In [ ]:
p = plot(;
    legend=:topleft,
    title="Total Wall Clock Time",
    ylabel="Time(s)",
    xlabel="Grid size",
    yscale=:log10,
    xscale=:log2,
    # xticks=grid_sizes
)
for (k,df) in enumerate(DataFrames.groupby(res_df, :AD))
    # println(df)
    make_label = true
    for rdf in DataFrames.groupby(df, [:Run,:Target])
        # @show rdf
        method = rdf[1,:AD]
        gs = rdf[:,:GridSize]
        mem_res = rdf[:,:Time]
        if make_label
            label = string(method)
            make_label = false
        else
            label = nothing
        end
        plot!(p, gs, mem_res, color=k, label=label)
        scatter!(p, gs, mem_res, color=k, label=nothing)
    end
end
display(p)

In [ ]:
ad_df = DataFrames.groupby(res_df, :AD)
p = plot(;
    legend=true,
    title="Number of Objective/Gradient Calls",
    ylabel="Number of Calls",
    xlabel="Grid size",
    # yscale=:log10,
    # xscale=:log10,
    # xticks=grid_sizes
)
for (k,df) in enumerate(ad_df)
    # println(df)
    method = df[1,:AD]
    gs = df[:,:GridSize]
    ncalls = df[:,:NumCalls]
    plot!(p, gs, ncalls, color=k, label=string(method))
    scatter!(p, gs, ncalls, color=k, label=nothing)
end
display(p)

In [ ]:
ad_df = DataFrames.groupby(res_df, :AD)
p = plot(;
    legend=:topleft,
    title="Wall Clock Time per Call",
    ylabel="Time(s)",
    xlabel="Grid size",
    yscale=:log10,
    xscale=:log2,
    # xticks=grid_sizes
)
for (k,df) in enumerate(DataFrames.groupby(res_df, :AD))
    # println(df)
    make_label = true
    for rdf in DataFrames.groupby(df, [:Run,:Target])
        # @show rdf
        method = rdf[1,:AD]
        gs = rdf[:,:GridSize]
        tm_res = rdf[:,:Time] ./ rdf[:,:NumCalls]
        if make_label
            label = string(method)
            make_label = false
        else
            label = nothing
        end
        plot!(p, gs, tm_res, color=k, label=label)
        scatter!(p, gs, tm_res, color=k, label=nothing)
    end
end
display(p)
# for (k,df) in enumerate(ad_df)
#     # println(df)
#     method = df[1,:AD]
#     grid_sizes = df[:,:GridSize]
#     times = df[:,:Time] ./ df[:,:NumCalls]
#     plot!(p, grid_sizes, times, color=k, label=string(method))
#     scatter!(p, grid_sizes, times, color=k, label=nothing)
# end

# Memory Usage

In [ ]:
function svd_dimensions(ngrid)
    k = log2(ngrid)
    m = 2^Int(ceil(k/2))
    n = 2^Int(floor(k/2))
    return (m,n)
end

function svd_mem(nsv, ngrid)
    # m = Int(sqrt(ngrid))
    # return nsv + 2*nsv*m
    (m,n) = svd_dimensions(ngrid)
    return nsv * (m + n + 1)
end

function svd_mem_sweep(ngrid)
    # m = Int(sqrt(ngrid))
    (m,n) = svd_dimensions(ngrid)
    nsvs = 1:min(m,n)
    mem_svd = zeros(Int, min(m,n))
    for nsv in nsvs
        mem_svd[nsv] = svd_mem(nsv, ngrid)
    end
    return (nsvs, mem_svd)
end

function svd_mem_savings()
    mem_savings = Dict{Int, Any}()
    for k in 2:14
        Nx = 2^k
        (nsvs, mem_svd) = svd_mem_sweep(Nx)
        saving = Nx * 8 / 1024 .- mem_svd * 8 / 1024
        mem_savings[Nx] = (nsvs, saving)
    end
    return mem_savings
end

function plot_savings(mem_save)
    (m,n) = svd_dimensions(maximum(keys(mem_save)))
    xmax = 0.5 * max(m,n)
    p = plot(
        # xscale=:log10,
        legend=true,
        ylabel="kB",
        xlabel="Number Singular Values",
        title="SVD compression vs standard (per step)",
        xlim=(0, xmax),
    )
    for Nx in sort(collect(keys(mem_save)))
        (nsvs, save) = mem_save[Nx]
        plot!(p, nsvs, save, label=string(Nx))
    end
    return p
end

In [ ]:
Nx = 1024
(num_svs, mem_svd) = svd_mem_sweep(Nx)
p = plot(num_svs, mem_svd * 8 / 1024,
    legend=false,
    ylabel="kB",
    xlabel="Number Singular Values",
    title="SVD compression vs standard (per step)",
)
plot!(p, num_svs, fill(Nx * 8 / 1024, length(num_svs)))
plot!(p, num_svs, fill(Nx * 8 / 1024, length(num_svs)) - mem_svd * 8 / 1024, label="Savings")

In [ ]:
mr = svd_mem_savings()
plot_savings(mr)

# SVD Effects on Solution

In [ ]:
function grid_control(i, xi, x, p)
    return x[i]
end

function tf_sin(x)
    return 0.5 + 0.2 * sin(2 * pi * x)
end
    
function initial_condition(x, p)

    Nx = p[:Nx]
    ic = p[:ic]
    u0 = zeros(eltype(x), Nx)

    for i in 1:Nx
        xi = BurgersEquation.gridpoint(i, Nx)
        u0[i] = ic(i, xi, x, p)
    end

    return u0

end

function burger_solution(
    x,
    p;
    save::Bool=false,
    progress::Bool=false,
)

    f(u) = 0.5 * u^2
    fu(u) = u

    u0 = initial_condition(x, p)
    Nx = p[:Nx]
    cfl = p[:cfl]
    tf = p[:tf]
    # Assumes u >= 0
    umax = maximum(u0)
    dx = BurgersEquation.gridsize(Nx)
    dt = 1.0 / ceil(tf * umax / (cfl * dx))

    save_rate = save ? max(1, Int(floor(2e-3 / dt))) : -1
    cfl = dt * umax / dx

    bp = BurgersEquation.setup(
        u0, f, fu, p[:tf], dt, Nx, p[:flux];
        save_rate=save_rate
    )
    BurgersEquation.solve(bp; progress=progress)

    return bp

end

In [ ]:
if compute == "optimization"
    Nx = 2^8
    cfl = 0.85
    tf = 1.0
    dx = BurgersEquation.gridsize(Nx)
    my_params = Dict(
        :Nx => Nx,
        :cfl => cfl,
        :tf => tf,
        :flux => :lf,
        :scale => 1e2,
        :ic => grid_control,
        # :target => tf_sin,
        :mode => :normal,
    )
    # x0 = fill(0.2, Nx)
    # x0 = [tf_sin(xk) for xk in 0.0:dx:1.0-0.5*dx]
    u0csv = joinpath(results_dir, make_base_name(:reverse, Nx, 1) * ".csv")
    df = CSV.read(u0csv, DataFrames.DataFrame, header=false)
    x0 = df[:,:Column1]
    # (m, n) = svd_dimensions(Nx)
    bp = burger_solution(x0, my_params; progress=true)
    (m,n) = svd_dimensions(Nx)
    @show Nx
    @show svd_dimensions(Nx)
end

In [ ]:
if compute == "optimization"
    u0 = bp.u0
    uf = bp.uk
    usvd0 = IAD.SVDVector(u0, m, n, 1)
    usvdf = IAD.SVDVector(uf, m, n, 1)
    p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
    plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
    q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
    # display(p)
    p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
    plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
    q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
    p = plot(p1, p2, q1, q2, size=(1000,600))
    # png(p, "/Users/jmaack/Desktop/SVDcompression")
    # display(p)
end

In [ ]:
if compute == "optimization"
    u0 = bp.u0
    uf = bp.uk
    usvd0 = IAD.SVDVector(u0, m, n, 1e-5)
    @show usvd0.nsv
    usvdf = IAD.SVDVector(uf, m, n, 1e-5)
    @show usvdf.nsv
    p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
    plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
    q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
    # display(p)
    p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
    plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
    q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
    plot(p1, p2, q1, q2, size=(1000,600))
    # display(p)
end

# Scratch

In [ ]:
function tf_weierstrass(x; a=0.85, b=7, N=3)
    vmax = 0.9
    vmin = 0.1
    vmid = 0.5 * (vmax + vmin)

    val = 0.0
    wbd = 0.0
    for n in 1:N
        val += a^n * cos(b^n * pi * x)
        wbd += a^n
    end

    val = 0.5 * (val + wbd) * (vmax - vmin) / wbd + vmin

    return val
end

In [ ]:
a = 0.825
b = 7
@show a * b
@show 1 + 3/2 * pi
;

In [ ]:
tf_weierstrass(0.0; a=a, b=b, N=5)

In [ ]:
tf_weierstrass(1.0; a=a, b=b, N=5)

In [ ]:
dx = 1e-6
xk = 0.0:dx:1.0
wek = tf_weierstrass.(xk; a=a, b=b, N=5)
plot(xk, wek)

In [ ]:
maximum(wek)

In [ ]:
minimum(wek)

In [ ]:
a + a^2 + a^3

In [ ]:
cfl = 0.85
tf = 1.0
p = plot()
q = plot()

for k in 6:-1:6
    Nx = 2^k
    dx = BurgersEquation.gridsize(Nx)
    my_params = Dict(
        :Nx => Nx,
        :cfl => cfl,
        :tf => tf,
        :flux => :lf,
        :scale => 1e2,
        :ic => grid_control,
        # :target => tf_sin,
        :mode => :normal,
    )
    xk = 0.0:dx:1.0
    x0 = tf_weierstrass.(xk; a=a, b=b, N=5)
    bp = burger_solution(x0, my_params; progress=true, save=true)
    plot!(
        p, 
        BurgersEquation.space_grid(Nx), 
        BurgersEquation.expand_solution(bp.u0);
        label="u0_$Nx",
    )
    plot!(
        q,
        BurgersEquation.space_grid(Nx),
        BurgersEquation.expand_solution(bp.uk);
        label="uT_$Nx",
    )
end
# (m,n) = svd_dimensions(Nx)
# @show Nx
# @show svd_dimensions(Nx)
pq = plot(p, q, size=(1000,400))
display(pq)
;

In [ ]:
cfl = 0.85
tf = 1.0
Nx = 2^8
dx = BurgersEquation.gridsize(Nx)
my_params = Dict(
    :Nx => Nx,
    :cfl => cfl,
    :tf => tf,
    :flux => :lf,
    :scale => 1e2,
    :ic => grid_control,
    # :target => tf_sin,
    :mode => :normal,
)
xk = 0.0:dx:1.0
x0 = tf_weierstrass.(xk; a=a, b=b, N=5)
bp = burger_solution(x0, my_params; progress=true, save=true)
;

In [ ]:
# make_gif(bp, "weierstrass"; fps=20)

In [ ]:
u0 = bp.u0
uf = bp.uk
(m, n) = svd_dimensions(length(u0))
usvd0 = IAD.SVDVector(u0, m, n, 1)
@show usvd0.nsv
usvdf = IAD.SVDVector(uf, m, n, 1)
@show usvdf.nsv
p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
# display(p)
p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
plot(p1, p2, q1, q2, size=(1000,600))
# display(p)

In [ ]:
u0 = bp.u0
uf = bp.uk
(m, n) = svd_dimensions(length(u0))
usvd0 = IAD.SVDVector(u0, m, n, 1e-5)
@show usvd0.nsv
usvdf = IAD.SVDVector(uf, m, n, 1e-5)
@show usvdf.nsv
p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
# display(p)
p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
plot(p1, p2, q1, q2, size=(1000,600))
# display(p)